In [23]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [24]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000908227s, CPU 0.000909s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000375413s, CPU 0.00037s)
Info    : Meshing 3D.

In [25]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:HeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321)

In [26]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [27]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [28]:
FEM.initialTemperature!("left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [29]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [30]:
FEM.applyBoundaryConditions!(K, C, q, [supp])

In [31]:
λmax = FEM.largestEigenValue(K, C)

29474.62441063031

In [32]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.2486392161077944e-5 … 0.006115173083554555 0.005655143580127581; 0.0 0.0002388769758174597 … 0.013060995910636931 0.012583050525767227], [0.0, 0.0019957346560979224, 0.003991469312195845, 0.005987203968293767, 0.00798293862439169, 0.009978673280489611, 0.011974407936587533, 0.013970142592685454, 0.015965877248783376, 0.017961611904881297, 0.01995734656097922, 0.02195308121707714, 0.023948815873175062, 0.025944550529272983, 0.027940285185370905, 0.029936019841468826, 0.03193175449756675, 0.033927489153664676, 0.0359232238097626, 0.037918958465860526], Int64[], 20, :T, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [33]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0008335803349351792 … 0.019271366519329328 0.020440644855610736; 0.0 0.004365347787506133 … 0.02804493083809825 0.029298012153639117], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [34]:
FEM.showDoFResults(T, :T, visible=true)

0

In [35]:
q1 = FEM.solveHeatFlux(T)

LowLevelFEM.VectorField([[0.0 -8.67631188906714e-15 … -1.3335568007227404e-6 -2.687352360112155e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 -4.4373425918681914e-30 … -2.9778502051908996e-22 5.955700410381799e-22; 0.0 4.4373425918681914e-30 … 5.955700410381799e-22 -5.955700410381799e-22], [0.0 -1.7791426939864793e-14 … -2.1186264475640967e-6 -4.155234312014839e-6; 0.0 -4.259848888193464e-28 … -2.3822801641527197e-21 0.0; … ; 0.0 -5.32481111024183e-29 … -4.7645603283054394e-21 2.3822801641527197e-21; 0.0 1.7749370367472766e-29 … 1.1911400820763599e-21 0.0], [0.0 -8.789233440404624e-14 … -5.548814773284402e-6 -1.0373077607971006e-5; 0.0 -3.9758589623138995e-27 … -1.9058241313221758e-20 -7.623296525288703e-20; … ; 0.0 -2.129924444096732e-28 … -1.9058241313221758e-20 -1.4293680984916318e-20; 0.0 -5.309063813275582e-28 … -5.6198706524954444e-21 -1.6292883476574588e-20], [0.0 -4.592722611852514e-13 … -1.5467988559098265e-5 -2.7640748303724845e-5; 0.0 -2.0447274663328626e-26 … 1.5246593050577406e-19 -3.811

In [36]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [37]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -8.676311889067161e-15 … -1.333556800722743e-6 -2.6873523601121547e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 7.806255641895632e-16 … -1.2490009027033011e-15 0.0; 0.0 -1.5612511283791264e-16 … 3.1225022567582528e-15 3.747002708109903e-15], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [38]:
FEM.showDoFResults(q2, :qvec)

2

In [39]:
q3 = FEM.elementsToNodes(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -8.676311889067161e-15 … -1.333556800722743e-6 -2.6873523601121547e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 7.806255641895632e-16 … -1.2490009027033011e-15 0.0; 0.0 -1.5612511283791264e-16 … 3.1225022567582528e-15 3.747002708109903e-15], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [40]:
FEM.showDoFResults(q3, :qvec)

3

In [41]:
err1 = FEM.fieldError(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D, LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321))

In [42]:
FEM.showDoFResults(err1, :vector)

4

In [43]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [44]:
gmsh.finalize()